In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('/Users/jacksonsorenson/Documents/Pyhton Projects/Seagate Revenue Optimization/Orginal CSV/supply_chain_data.csv')
df.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [9]:
# Beginning Data Cleaning
# Check for missing values
print("\nMissing Values Summary:")
print(df.isnull().sum())
print(df.dtypes)

# No null values detected


Missing Values Summary:
Product type               0
SKU                        0
Price                      0
Availability               0
Number of products sold    0
Revenue generated          0
Customer demographics      0
Stock levels               0
Lead times                 0
Order quantities           0
Shipping times             0
Shipping carriers          0
Shipping costs             0
Supplier name              0
Location                   0
Lead time                  0
Production volumes         0
Manufacturing lead time    0
Manufacturing costs        0
Inspection results         0
Defect rates               0
Transportation modes       0
Routes                     0
Costs                      0
dtype: int64
Product type                object
SKU                         object
Price                      float64
Availability                 int64
Number of products sold      int64
Revenue generated          float64
Customer demographics       object
Stock levels         

In [10]:
# Viewing data types for each column within df, better view of columns
print(df.dtypes)

Product type                object
SKU                         object
Price                      float64
Availability                 int64
Number of products sold      int64
Revenue generated          float64
Customer demographics       object
Stock levels                 int64
Lead times                   int64
Order quantities             int64
Shipping times               int64
Shipping carriers           object
Shipping costs             float64
Supplier name               object
Location                    object
Lead time                    int64
Production volumes           int64
Manufacturing lead time      int64
Manufacturing costs        float64
Inspection results          object
Defect rates               float64
Transportation modes        object
Routes                      object
Costs                      float64
dtype: object


In [11]:
# Drop duplicates if present
df = df.drop_duplicates()

In [13]:

#Feature Engineering

# Drop the 'High Defect Rate' column from the main dataset
df = df.drop(columns=['High Defect Rate'], errors='ignore')

# Verify that the column has been dropped


# Profit Margin
df['Profit Margin'] = df['Revenue generated'] - df['Costs']

# Calculate Stock Coverage (Stock levels divided by order quantities)
df['Stock Coverage'] = df['Stock levels'] / df['Order quantities']

# Add a Flag for High Defect Rates (greater than 0.2)
df['High Defect Rate'] = np.where(df['Defect rates'] > 0.2, 1, 0)

# Transportation Mode Efficiency
transport_costs = df.groupby('Transportation modes')['Costs'].mean().to_dict()
df['Transportation Efficiency'] = df['Transportation modes'].map(transport_costs)

# Price Tier (Low, Medium, High) based on quantiles
df['Price Tier'] = pd.qcut(df['Price'], q=3, labels=['Low', 'Medium', 'High'])

# Cost per product sold
df['Cost per Product Sold'] = df['Costs'] / (df['Number of products sold'] + 1e-6)

# Defects Per product type
df['Defects per Product'] = df['Defect rates'] * df['Number of products sold']

# Lesd Time efficiency
df['Lead Time Efficiency'] = df['Lead times'] / (df['Stock levels'] + 1e-6)

# Lead time cost
df['Cost × Lead Time'] = df['Costs'] * df['Lead times']

# Manufacturing Cost
df['Manufacturing Cost × Defect Rate'] = df['Manufacturing costs'] * df['Defect rates']

# Low stock Warning threshold
df['Low Stock Warning'] = (df['Stock levels'] < 10).astype(int)

# Log cost
df['Log Costs'] = np.log1p(df['Costs'])

# Displaying the cleaned dataset with features
print("\nCleaned Data with New Features:")
print(df.head())

# Save the cleaned dataset
output_file_path = 'cleaned_supply_chain_data.csv'  
df.to_csv(output_file_path, index=False)
print(f"\nCleaned data saved to {output_file_path}")



Cleaned Data with New Features:
  Product type   SKU      Price  Availability  Number of products sold  \
0     haircare  SKU0  69.808006            55                      802   
1     skincare  SKU1  14.843523            95                      736   
2     haircare  SKU2  11.319683            34                        8   
3     skincare  SKU3  61.163343            68                       83   
4     skincare  SKU4   4.805496            26                      871   

   Revenue generated Customer demographics  Stock levels  Lead times  \
0        8661.996792            Non-binary            58           7   
1        7460.900065                Female            53          30   
2        9577.749626               Unknown             1          10   
3        7766.836426            Non-binary            23          13   
4        2686.505152            Non-binary             5           3   

   Order quantities  ...  Transportation Efficiency Price Tier  \
0                96  ..